In [1]:
from IPython.display import display, HTML
display(HTML('<link rel="preconnect" href="https://fonts.googleapis.com">\
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>\
<link href="https://fonts.googleapis.com/css2?family=Comfortaa:wght@300&display=swap" rel="stylesheet">'))

<font face="Comfortaa"><h1> British Airways

<font face="Comfortaa"><h2> 1. Data Collection - Web Scraping

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set(rc={'axes.facecolor':'black'})
sns.set_palette("pastel")

In [4]:
url = "https://www.airlinequality.com/airline-reviews/british-airways"
response = requests.get(url)

if response.status_code == 200:
    html_content = response.content
else:
    print("Failed to retrieve the page")

In [5]:
soup = BeautifulSoup(html_content, 'html.parser')

In [6]:
navigation_bar = soup.find_all('article', class_='comp comp_reviews-pagination querylist-pagination position-')
if navigation_bar:
    navigation_bar = navigation_bar[-1]
    li_element = navigation_bar.find_all('li')
    if li_element:
        last_li = li_element[-1]
        a_element = last_li.find('a')
        link = a_element['href']

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_reviews(url):
    reviews = []

    while url:
        response = requests.get(url)
        
        if response.status_code == 200:
            html_content = response.content
            soup = BeautifulSoup(html_content, 'html.parser')
            
            for article in soup.find_all('article'):
                review = article.find('div', class_='body')
                if review:
                    review_id = review['id']
                    review_header = review.find('h2', class_='text_header').text
                    review_author_location_date = review.find('h3').text
                    review_content = review.find('div', class_='tc_mobile')
                    review_body = review_content.find('div', class_='text_content').text
                    review_rating_table = review_content.find('table', class_='review-ratings')
                    for tr in review_rating_table.find_all('tr'):
                        td_elements = tr.find_all('td')
                        if len(td_elements) >= 2:
                            rating_name = td_elements[0].text
                            rating_value_td = td_elements[1]
                            rating_value = rating_value_td.text

                            span_elements = rating_value_td.find_all('span', class_='star fill')
                            count_fill_spans = len(span_elements)
                            reviews.append({
                            'review_id': review_id,
                            'review_header': review_header,
                            'review_author_location_date': review_author_location_date,
                            'review_body': review_body,
                            'rating_name': rating_name,
                            'rating_value': count_fill_spans if span_elements else rating_value
                            })

            
            navigation_bar = soup.find_all('article', class_='comp comp_reviews-pagination querylist-pagination position-')
            if navigation_bar:
                navigation_bar = navigation_bar[-1]
                li_element = navigation_bar.find_all('li')
                if li_element:
                    last_li = li_element[-1]
                    a_element = last_li.find('a')
                    if a_element:
                        url = a_element['href']
                        url = "https://www.airlinequality.com/"+url
                        print(url)
                    else:
                        url = None
                else:
                    url = None
            else:
                url = None
                
        else:
            print(f"Failed to retrieve the page: {url}")
            break
    
    return reviews

In [8]:
start_url = "https://www.airlinequality.com/airline-reviews/british-airways"
all_reviews = scrape_reviews(start_url)


https://www.airlinequality.com//airline-reviews/british-airways/page/2/
https://www.airlinequality.com//airline-reviews/british-airways/page/3/
https://www.airlinequality.com//airline-reviews/british-airways/page/4/
https://www.airlinequality.com//airline-reviews/british-airways/page/5/
https://www.airlinequality.com//airline-reviews/british-airways/page/6/
https://www.airlinequality.com//airline-reviews/british-airways/page/7/
https://www.airlinequality.com//airline-reviews/british-airways/page/8/
https://www.airlinequality.com//airline-reviews/british-airways/page/9/
https://www.airlinequality.com//airline-reviews/british-airways/page/10/
https://www.airlinequality.com//airline-reviews/british-airways/page/11/
https://www.airlinequality.com//airline-reviews/british-airways/page/12/
https://www.airlinequality.com//airline-reviews/british-airways/page/13/
https://www.airlinequality.com//airline-reviews/british-airways/page/14/
https://www.airlinequality.com//airline-reviews/british-air

https://www.airlinequality.com//airline-reviews/british-airways/page/115/
https://www.airlinequality.com//airline-reviews/british-airways/page/116/
https://www.airlinequality.com//airline-reviews/british-airways/page/117/
https://www.airlinequality.com//airline-reviews/british-airways/page/118/
https://www.airlinequality.com//airline-reviews/british-airways/page/119/
https://www.airlinequality.com//airline-reviews/british-airways/page/120/
https://www.airlinequality.com//airline-reviews/british-airways/page/121/
https://www.airlinequality.com//airline-reviews/british-airways/page/122/
https://www.airlinequality.com//airline-reviews/british-airways/page/123/
https://www.airlinequality.com//airline-reviews/british-airways/page/124/
https://www.airlinequality.com//airline-reviews/british-airways/page/125/
https://www.airlinequality.com//airline-reviews/british-airways/page/126/
https://www.airlinequality.com//airline-reviews/british-airways/page/127/
https://www.airlinequality.com//airlin

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

<font face="Comfortaa"><h2>2. Store Data into Dataframe

In [ ]:
reviews_df = pd.DataFrame(all_reviews)
# reviews_df.to_csv('airline_reviews.csv', index=False)

In [ ]:
reviews_df = reviews_df.drop_duplicates()

In [ ]:
reviews_df.head()

<font face="Comfortaa"><h2>3. Data Cleaning

In [ ]:
reviews_df = reviews_df.pivot(index=['review_id', 'review_header', 'review_author_location_date', 'review_body'], columns='rating_name', values='rating_value').reset_index()

In [ ]:
reviews_df.info()

In [ ]:
reviews_df.head()

In [ ]:
reviews_df['review_author_location_date'] = reviews_df['review_author_location_date'].str.replace('\n\n', '')
split_cols = reviews_df['review_author_location_date'].str.split(r'\(|\)', expand = True)

In [ ]:
reviews_df['author'] = split_cols[0]
reviews_df['location'] = split_cols[1]
reviews_df['date'] = split_cols[2]

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.describe(include='all')

In [ ]:
reviews_df = reviews_df.drop(axis=1, columns=['review_author_location_date'])

In [ ]:
reviews_df.info()

In [ ]:
reviews_df.columns = ['id', 'header', 'body', 'aircraft', 'cabin_staff_service', 'flown_date', 'fnb',\
                       'ground_service', 'inflight_entertainment', 'recommended', 'route', 'seat_comfort', \
                       'seat_type', 'traveller_type', 'value_for_money', 'wifi', 'author', 'location', 'review_date']

In [ ]:
reviews_df.head()

In [ ]:
reviews_df['fnb'] = reviews_df['fnb'].replace('N/A', np.nan)
reviews_df['inflight_entertainment'] = reviews_df['inflight_entertainment'].replace('N/A', np.nan)
reviews_df['seat_comfort'] = reviews_df['seat_comfort'].replace('N/A', np.nan)
reviews_df['value_for_money'] = reviews_df['value_for_money'].replace('N/A', np.nan)

In [ ]:
#float
reviews_df['cabin_staff_service'] = reviews_df['cabin_staff_service'].astype('float')
reviews_df['ground_service'] = reviews_df['ground_service'].astype('float')
reviews_df['fnb'] = reviews_df['fnb'].astype('float')
reviews_df['inflight_entertainment'] = reviews_df['inflight_entertainment'].astype('float')
reviews_df['seat_comfort'] = reviews_df['seat_comfort'].astype('float')
reviews_df['value_for_money'] = reviews_df['value_for_money'].astype('float')
reviews_df['wifi'] = reviews_df['wifi'].astype('float')

#date
reviews_df['flown_date'] = pd.to_datetime(reviews_df['flown_date'], errors='coerce')
reviews_df['review_date'] = pd.to_datetime(reviews_df['review_date'], errors='coerce')


In [ ]:
reviews_df.head()

<font face="Comfortaa"><h2>4. Exploratory Data Analysis

<font face="Comfortaa"><h3>4.1 Statistical Analysis

In [ ]:
reviews_df.describe()

In [ ]:
reviews_df['cabin_staff_service'].value_counts(normalize=True)

In [ ]:
numerical_variables = reviews_df.describe().columns.to_list()
title_names = ['Cabin Staff Service', 'Food & Beverage', 'Ground Service', 'Inflight Entertainment', 'Seat Comfort', 'Value of Money', 'WIFI & Connectivity']

fig, ax = plt.subplots(ncols=2, nrows=4, figsize=(10, 15))

for n, var in enumerate(numerical_variables):
    i = n // 2
    j = n % 2
    sns.histplot(data=reviews_df, x=var, ax=ax[i, j])
    ax[i, j].set(xlabel='Frequency', ylabel='', title=title_names[n])
    ax[i, j].grid(False)

    plt.tight_layout()
plt.show()

<font face="Comfortaa">
    <h4><b> Statistical Summary: <b></h4> 
    <b>WIFI & Connectivity </b> received the highest percentage of 1-star reviews  & lowest median at 1.93.<br> 
    <b>Cabin Staff Service </b> received the highest percentage of 5-star reviews & highest median at 3.24.<br>
    <b>Seat Comfort</b> received relatively equal numbers of reviews throughout 1 to 5 stars & second highest median at 2.87. <br>
        All of the other review type including <b>Food & Beverage</b>, <b>Ground Service</b>, <b>Inflight Entertainment</b>, <b>Value of Money</b> although receieved more 1 star review but the medians range between 2.65 to 2.80.  
 </font>

<font face="Comfortaa"><h3>4.2 Sentimental Analysis

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
def get_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    compound_score = sentiment['compound']
    
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [ ]:
reviews_df['sentiment'] = reviews_df['body'].apply(get_sentiment)

In [ ]:
reviews_df['sentiment'] = reviews_df['sentiment'].astype('category')
reviews_df['sentiment'] = reviews_df['sentiment'].cat.set_categories(['Positive', 'Negative', 'Neutral'])

In [ ]:
sns.countplot(data = reviews_df,\
            x = 'sentiment')
plt.show()

In [ ]:
reviews_df['recommended'].value_counts()

In [ ]:
reviews_df.groupby(['recommended','sentiment'])['id'].count()

<font face="Comfortaa">
    <h4><b> Sentimental Summary: </b></h4> 
    In conducting sentiment analysis using VADER, the majority of the reviews were found to be <b>Positive</b>," followed by <b>"Negative"</b> with a relatively small number categorized as <b>"Neutral" </b>. <br> However, this sentiment analysis appears to contradict the data in the <b>recommended</b> column, where a significant portion of reviewers did not recommend booking with the airline.<b>British Airways</b>. By looking closer to these columns, most of the sentiment were aligned with recommended column.
 </font>

<font face="Comfortaa"><h3>4.3 Time Series Analysis

In [ ]:
reviews_df['date_diff'] = reviews_df['review_date'] - reviews_df['flown_date']

In [ ]:
reviews_df['review_month'] = reviews_df['review_date'].dt.month
reviews_df['flown_month'] = reviews_df['flown_date'].dt.month

In [ ]:
reviews_df['recommended'] = reviews_df['recommended'].astype('category')
reviews_df['recommended'] = reviews_df['recommended'].cat.set_categories(['yes','no'])

In [ ]:
date_cols = ['review_month', 'flown_month']
title_name = ['Review by Month', 'Flown by Month']
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(10, 3), sharey=True)

for n, var in enumerate(date_cols):
    sns.countplot(data=reviews_df, x=var, ax=ax[n], hue = 'recommended')
    ax[n].set(xlabel='Frequency', ylabel='', title=title_name[n])
    ax[n].grid(False)
    ax[n].legend(loc='center left', bbox_to_anchor=(1, 0.5), facecolor='k', labelcolor='w')
plt.tight_layout()
plt.show()

In [ ]:
reviews_df['review_month'].value_counts()

<font face='Comfortaa'>
    <b>Time Series Summary:</b><br>
    In the course of the time series analysis conducted on review dates and flown dates within the dataset, it was observed that not all reviews were paired with corresponding flown dates, indicating potential data gaps.<br>

The distribution of reviews across months revealed that June had the highest number of reviews, totaling 354, followed closely by October with 339 reviews. These two months stood out as periods of increased customer feedback.<br>

A noteworthy trend that emerged across all months was that a significant proportion of reviewers chose not to recommend the airline. This consistent sentiment of dissatisfaction suggests that a substantial segment of customers had reservations about recommending the airline during this period.<br>
    
</font>

<font face="Comfortaa"><h3>4.4 Word Cloud

In [ ]:
from wordcloud import WordCloud
text_data = reviews_df['body'].str.cat(sep=' ')
wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=100).generate(text_data)

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

<font face='Comfortaa'>
    <b>WordCloud Summary:</b><br>
    It appears to cover a range of aspects, including airport experiences, cabin service, delays, and more.
    
</font>